In [ ]:
import math
import numpy as np
import awkward as ak
import time

In [ ]:
# for the local model
import torch

torch.hub._validate_not_a_forked_repo = lambda a, b, c: True
local_model = torch.hub.load("pytorch/vision:v0.10.0", "resnet50", pretrained=True).eval()

In [ ]:
from utils.tritonutils import wrapped_triton

# create instance of triton model
triton_model = wrapped_triton( "triton+grpc://test-3.apps.okddev.fnal.gov:443/resnet50_torch/1")


In [ ]:
# create random data
data = np.random.rand(10,3,224,224).astype(np.float32)

In [ ]:
# test local
with torch.no_grad():
    local_output = local_model(torch.from_numpy(data))#.detach().numpy()
print(local_output)

In [ ]:
# test triton
out = triton_model({'input__0':data},'output__0')
out

In [ ]:
# create random inputs to test
n_inputs = 2000
test_inputs = {'input': np.random.rand(n_intputs,3,224,224).astype(np.float32)}
test_inputs_ak = ak.Array(test_inputs)

In [ ]:
def process_jets(in_jets, batch_size=1024, use_triton=False):
    
    print('Running triton server inference' if use_triton else 'Running local inference')
    
    # define variables to track processing time
    njets = np.array([])
    t = np.array([])
    t_begin = time.time()
    
    # loop through input data batches and run inference on each batch
    for ii in range(0, len(in_jets), batch_size):
        print('%i/%i jets processed, processing next batch'%(ii,len(in_jets)))

        # get a batch of data
        try:
            jets_eval = in_jets[ii:ii + batch_size]
            njets = np.append(njets, ii+batch_size)
        except:
            jets_eval = in_jets[ii:-1]
            njets = np.append(njets, len(in_jets))

        ## structure inputs slightly differently and run inference depending on model
        # triton model
        if use_triton:
            X = {}
            c = 0
            for k in jets_eval.fields:
                X[f'{k}__{c}'] = ak.to_numpy(jets_eval[k])
                c += 1
                
            # triton inference
            outputs = triton_model(X)
                
        # local model   
        else:
            for k in jets_eval.fields:
                X = ak.to_numpy(jets_eval[k])
                
            # local inference
            with torch.no_grad():
                local_output = local_model(torch.from_numpy(X))

        t = np.append(t, time.time()-t_begin)
        
    print('Total time elapsed = %f sec'%t[-1])

    return njets, t
    

In [ ]:
# to minimize noise due to connection with triton server, take avg results of n trials
n = 3
local_t = None
for ii in range(n):
    print('------ Trial %i ------'%ii)
    if local_t is None:
        local_njets, local_t = process_jets(test_inputs_ak, use_triton=False, batch_size=256)
    else:
        local_njets, local_t_temp = process_jets(test_inputs_ak, use_triton=False, batch_size=256)
        local_t += local_t_temp
local_t /= n

In [ ]:
# to minimize noise due to connection with triton server, take avg results of n trials
n = 5
triton_t = None
for ii in range(n):
    print('------ Trial %i ------'%ii)
    if triton_t is None:
        triton_njets, triton_t = process_jets(test_inputs_ak, use_triton=True, batch_size=256)
    else:
        triton_njets, triton_t_temp = process_jets(test_inputs_ak, use_triton=True, batch_size=256)
        triton_t += triton_t_temp
triton_t /= n

## now plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import mplhep as hep
plt.style.use(hep.style.CMS)

In [ ]:
fig = plt.figure()
# set height ratios for subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1])

# the first subplot
ax0 = plt.subplot(gs[0])
# log scale for axis Y of the first subplot
ax0.set_yscale("log")
line0, = ax0.plot(local_njets, local_t, color='r')
line1, = ax0.plot(triton_njets, triton_t, color='b')

# the second subplot
# shared axis X
ax1 = plt.subplot(gs[1], sharex = ax0)
line2, = ax1.plot(local_njets, local_t/triton_t, color='black', linestyle='--')
plt.setp(ax0.get_xticklabels(), visible=False)
# remove last tick label for the second subplot
yticks = ax1.yaxis.get_major_ticks()
yticks[-1].label1.set_visible(False)

# put legend on first subplot
ax0.legend((line0, line1), ('local model', 'triton model'), loc='lower right')

ax0.set_ylabel('time elapsed (s)')
ax1.set_ylabel('$t_{local}/t_{triton}$')
ax1.set_xlabel('# inputs processed')

# remove vertical gap between subplots
plt.subplots_adjust(hspace=.0)
plt.rcParams["figure.figsize"] = (7,6)
plt.savefig("results/timing_results_resnet50.eps", bbox_inches="tight")
plt.show()